In [1]:
using DataFrames

In [2]:
cur_table = readtable("../data/tokamak_db.csv")

cur_table[:SELDB3] = map(Int,cur_table[:SELDB3])

return

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #readtable#232(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at /Users/dan/.julia/v0.6/DataFrames/src/deprecated.jl:1045
 [3] readtable(::String) at /Users/dan/.julia/v0.6/DataFrames/src/deprecated.jl:1045
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] include_string(::Module, ::String, ::String) at /Users/dan/.julia/v0.6/Compat/src/Compat.jl:88
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/dan/.julia/v0.6/IJulia/src/execute_request.jl:180
 [7] (::Compat.#inner#14{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/dan/.julia/v0.6/Compat/src/Compat.jl:332
 [8] eventloop(::ZMQ.Socket) at /Users/dan/.julia/v0.6/IJulia/src/eventloop.jl:8
 [

In [3]:
cur_func = function(aa,nn,ii,rr,bb,AA,PL,kk,ss)
    ( ss == 1111111111 ) || return NaN
    
    cur_val = NaN
    try
        cur_val = ( 0.145 * (abs(ii)/1e6)^0.93 * rr ^ 1.39 * aa ^ 0.58 * kk ^ 0.78 * (nn/1e20) ^ 0.41 * abs(bb) ^ 0.15 * AA ^ 0.19 / (PL/1e6) ^ 0.69)
        ismissing(cur_val) && return NaN
        @assert cur_val > 0
        return cur_val
    catch cur_error
        println(cur_error)
        println(cur_val)
        println((aa,nn,ii,rr,bb,AA,PL,kk,ss))
        return NaN
    end
end

cur_table[:TAUE] = map(
    cur_func, 
    cur_table[:AMIN], cur_table[:NEL], cur_table[:IP], 
    cur_table[:RMAG], cur_table[:BT], cur_table[:MEFF], 
    cur_table[:PLTH], cur_table[:KAREA], cur_table[:SELDB3]
)

return

In [4]:
using Interact
using DataStructures

using StringCases
using LaTeXStrings
using Query

using Plots
pgfplots()

Plots.PGFPlotsBackend()

In [106]:
cur_query = @from cur_row in cur_table begin
    @where !isnull(cur_row.TAUE) && !isnull(cur_row.TAUTOT) && !isnan(cur_row.TAUE)
    @where cur_row.SELDB3 == 1111111111
    @select cur_row.TAUE
    @collect
end

cur_fit_tau = Array(cur_query)

cur_query = @from cur_row in cur_table begin
    @where !isnull(cur_row.TAUE) && !isnull(cur_row.TAUTOT) && !isnan(cur_row.TAUE)
    @where cur_row.SELDB3 == 1111111111
    @select cur_row.TAUTOT
    @collect
end

cur_table_tau = Array(cur_query)

cur_plot = scatter(cur_table_tau, cur_fit_tau, opacity=0.4, markersize=0.65, label="")

cur_min = min(minimum(cur_fit_tau), minimum(cur_table_tau)) / 1.4
cur_max = max(maximum(cur_fit_tau), maximum(cur_table_tau)) * 1.4

plot!([cur_min, cur_max], [cur_min, cur_max], label="")

plot!(aspectratio=:equal, xscale=:log10, yscale=:log10)
xlabel!(L"(\tau_E)_{ TABLE } \ \ [ \textnormal{s} ]")
ylabel!(L"(\tau_E)_{ FIT } \ \ [ \textnormal{s} ]")

title!("Confinement Time Scaling Fit")

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 Z M 2.9375 -6.375 "/>
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 Z M 2.484375 0 "/>
 
 
<path style="stroke:none;" d="M 3.296875 2.390625 C 3.296875 2.359375 3.296875 2.34375 3.125 2.171875 C 1.890625 0.921875 1.5625 -0.96875 1.5625 -2.5 C 1.5625 -4.234375 1.9375 -5.96875 3.171875 -7.203125 C 3.296875 -7.328125 3.296875 -7.34375 3.296875 -7.375 C 3.296875 -7.453125 3.265625 -7.484375 3.203125 -7.484375 C 3.09375 -7.484375 2.203125 -6.796875 1.609375 -5.53125 C 1.109375 -4.4375 0.984375 -3.328125 0.984375 -2.5 C 0.984375 -1.71875 1.09375 -0.515625 1.640625 0.625 C 2.25 1.84375 3.09375 2.5 3.203125 2.5 C 3.265625 2.5 3.296875 2.46875 3.296875 2.390625 Z M 3.296875 2.390625 "/>
 
 
<path style="stroke:none;" d="M 2.875 -2.5 C 2.875 -3.265625 2.765625 -4.46875 2.21875 -5.609375 C 1.625 -6.828125 0.765625 -7.484375 0.671875 -7.484375 C 0.609375 -7.484375 0.5625 -7.4375 0.5625 -7.375 C 0.5625 -7.34375 0.5625 -7.328125 0.75 -7.140625 C 1.734375 -6.15625 2.296875 -4.578125 2.296875 -2.5 C 2.296875 -0.78125 1.9375 0.96875 0.703125 2.21875 C 0.5625 2.34375 0.5625 2.359375 0.5625 2.390625 C 0.5625 2.453125 0.609375 2.5 0.671875 2.5 C 0.765625 2.5 1.671875 1.8125 2.25 0.546875 C 2.765625 -0.546875 2.875 -1.65625 2.875 -2.5 Z M 2.875 -2.5 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 3.515625 -1.265625 L 3.28125 -1.265625 C 3.265625 -1.109375 3.1875 -0.703125 3.09375 -0.640625 C 3.046875 -0.59375 2.515625 -0.59375 2.40625 -0.59375 L 1.125 -0.59375 C 1.859375 -1.234375 2.109375 -1.4375 2.515625 -1.765625 C 3.03125 -2.171875 3.515625 -2.609375 3.515625 -3.265625 C 3.515625 -4.109375 2.78125 -4.625 1.890625 -4.625 C 1.03125 -4.625 0.4375 -4.015625 0.4375 -3.375 C 0.4375 -3.03125 0.734375 -2.984375 0.8125 -2.984375 C 0.96875 -2.984375 1.171875 -3.109375 1.171875 -3.359375 C 1.171875 -3.484375 1.125 -3.734375 0.765625 -3.734375 C 0.984375 -4.21875 1.453125 -4.375 1.78125 -4.375 C 2.484375 -4.375 2.84375 -3.828125 2.84375 -3.265625 C 2.84375 -2.65625 2.40625 -2.1875 2.1875 -1.9375 L 0.515625 -0.265625 C 0.4375 -0.203125 0.4375 -0.1875 0.4375 0 L 3.3125 0 Z M 3.515625 -1.265625 "/>
 
 
<path style="stroke:none;" d="M 3.59375 -2.21875 C 3.59375 -2.984375 3.5 -3.546875 3.1875 -4.03125 C 2.96875 -4.34375 2.53125 -4.625 1.984375 -4.625 C 0.359375 -4.625 0.359375 -2.71875 0.359375 -2.21875 C 0.359375 -1.71875 0.359375 0.140625 1.984375 0.140625 C 3.59375 0.140625 3.59375 -1.71875 3.59375 -2.21875 Z M 1.984375 -0.0625 C 1.65625 -0.0625 1.234375 -0.25 1.09375 -0.8125 C 1 -1.21875 1 -1.796875 1 -2.3125 C 1 -2.828125 1 -3.359375 1.09375 -3.734375 C 1.25 -4.

In [37]:
savefig(cur_plot, "h_mode.tex")